## 1. Calculate the gene expression score

In [ ]:
import os,gc
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np

os.chdir("/data/wuqinhua/phase/age")

In [ ]:
## ----------------- TOP100 gene ---------------------

adata = sc.read('./all_pbmc_anno_s.h5ad')
print("read_over")

age_gene = pd.read_csv("./Analysis_result/Attr_result/attr_age_all.csv") 
top_100 = age_gene.sort_values(by='attr_value', ascending=False).head(100)
top_100 = top_100['gene_name'].tolist()

bottom_100 = age_gene.sort_values(by='attr_value', ascending=True).head(100)
bottom_100 = bottom_100['gene_name'].tolist()

print("top100: ",top_100)
print("boottom100: ", bottom_100)
print("gene_over")


sc.tl.score_genes(adata, top_100, ctrl_size=100, gene_pool=None, n_bins=25, score_name='gene_ex_score', random_state=0, copy=False, use_raw=None)
adata.obs.to_csv('./Analysis_result/Conjoint_result/gene_ex_scores_top100.csv')
sc.tl.score_genes(adata, bottom_100, ctrl_size=100, gene_pool=None, n_bins=25, score_name='gene_ex_score', random_state=0, copy=False, use_raw=None)
adata.obs.to_csv('./Analysis_result/Conjoint_result/gene_ex_scores_bottom100.csv')

print("over")

## 2. Correlation scatter

In [ ]:
setwd("/data/wuqinhua/phase/age")
rm(list = ls())
gc()

library(cowplot) 
library(ggplot2) 
library(RColorBrewer) 
library(dplyr) 
library(tidyr)
library(ggpubr) 
library(broom)
library(tidyverse)
library(tibble)
library(janitor)
library(ggrepel)
library(tidyr)
library(ggplot2)
library(forestploter)
library(gridExtra)
library(tidyverse)
library(dplyr)
library(broom)
library(ggpubr)
library(randomForest)
library(mice)
library(reshape2)
library(gghalves)
library(cowplot)
library(patchwork)

In [ ]:
ex_t <- read.csv("'./Analysis_result/Conjoint_result/gene_ex_scores_top100.csv", row.names = 1)  
attnData = read.csv('./Analysis_result/Attn_result/attn_age_cell_PHASE.csv')
nameAll = unique(attnData$celltype)
nameList = sort(nameAll)

sampleFold = data.frame(id = character(), celltype = character(), fold = numeric())
idList = unique(attnData$Tube_id)
for (id in idList) {
  attnTmp = attnData %>% filter(Tube_id == id)
  avgScore = 1 / dim(attnTmp)[1]
  foldRes = attnTmp %>% group_by(celltype) %>% summarise(res = median(log2(attn/avgScore)))
  dataTmp = data.frame(id = rep(id,dim(foldRes)[1]),
                       celltype = foldRes$celltype,
                       fold = foldRes$res)
  dataTmp_s = dataTmp %>% filter(celltype %in% nameList)
  dataTmp_s$fold = scale(dataTmp_s$fold)
  sampleFold = rbind(sampleFold,dataTmp_s)
}

sampleFold.Table = dcast(sampleFold,id ~ celltype)
rownames(sampleFold.Table) = sampleFold.Table$id
sampleFold.Table$id = NULL

attn = sampleFold.Table

In [1]:
exData = ex_t  

nameAll = unique(exData$celltype)
nameList = sort(nameAll)

sampleFold = data.frame(id = character(), celltype = character(), fold = numeric())
idList = unique(exData$Tube_id)
for (id in idList) {
  attnTmp = exData %>% filter(Tube_id == id)
  avgScore = 1 / dim(attnTmp)[1]
  foldRes = attnTmp %>% group_by(celltype) %>% summarise(res = median(gene_ex_score))
  dataTmp = data.frame(id = rep(id,dim(foldRes)[1]),
                       celltype = foldRes$celltype,
                       fold = foldRes$res)
  dataTmp_s = dataTmp %>% filter(celltype %in% nameList)
  sampleFold = rbind(sampleFold,dataTmp_s)
}

ex_sampleFold.Table = dcast(sampleFold,id ~ celltype)
rownames(ex_sampleFold.Table) = ex_sampleFold.Table$id
ex_sampleFold.Table$id = NULL
ex = ex_sampleFold.Table
head(ex)

In [ ]:
sampleInfo = read.csv('./Info/sample_info.csv')
rownames(sampleInfo) = sampleInfo$Tube_id
sampleInfo = sampleInfo[rownames(attn),]
ex = ex[rownames(attn),]

label_df <- data.frame(sample = sampleInfo$Tube_id, label = sampleInfo$Age_group)

custom_colors <- c("A" = "#FFCCCC", "B" = "#999933", "C" = "#B0E57C", "D" = "#99CCFF", "E" = "#D2B5E1")

plot_list <- list()

for (celltype in colnames(ex_sampleFold.Table)) {

  merged_data <- data.frame(
    sample = rownames(attn),
    attn = attn[, celltype],
    ex = ex[, celltype],   
    label = label_df$label
  )

p <- ggscatter(merged_data, x = 'attn', y = 'ex', title = paste("Cell type:", celltype),
                 add = "reg.line", color = 'label',
                 add.params = list(color = "#4B4B4B", fill = "lightgray"),
                 cor.coeff.args = list(method = "pearson", size = 6),  
                 conf.int = TRUE, cor.coef = TRUE, cor.method = "pearson",  
                 size = 3, font.label = c(5, 'plain')) +
                 scale_color_manual(values = custom_colors)
  
 
  plot_list[[celltype]] <- p
  # ggsave(paste0("./Plot/Attn_plot//scatter_top100/correlation_", celltype, ".pdf"), plot = p, width = 5, height = 4)
}

combine_plot = wrap_plots(plot_list,ncol=8)
ggsave("./Plot/Attn_plot/scatter_ex_attn/correlation_top100.png",combine_plot,width = 40,height = 28)